## Data parsing

Essentially a reduced version of *create_edgelist.ipynb*

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import re

import glob, os
import itertools
from tqdm import tqdm

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
from shapely.geometry import LineString

Load raw data

In [2]:
dataset_folder = '../dataset/'			# will be used as output folder as well

members = pd.read_csv(dataset_folder + 'HSall_members.csv')
party_codes = pd.read_csv(dataset_folder + 'HSall_parties.csv')

In [3]:
def create_members_df():
    temp_congress = members.groupby('icpsr', as_index=False)[['congress']].agg(lambda x: list(x))
    temp_party = members.groupby('icpsr', as_index=False)[['party_code']].agg(lambda x: list(set(x)))
    temp_congress = temp_congress.merge(temp_party)
    temp_congress['bioname'] = temp_congress['icpsr'].map(members[['icpsr', 'bioname']].set_index('icpsr').to_dict()['bioname'])
    temp_congress['state_abbrev'] = temp_congress['icpsr'].map(members[['icpsr', 'state_abbrev']].set_index('icpsr').to_dict()['state_abbrev'])
    party_codes_dic = party_codes[['party_name', 'party_code']].set_index('party_code').to_dict()['party_name']
    temp_congress['party_name'] = temp_congress['party_code'].apply(lambda x: [party_codes_dic[y] for y in x])
    return temp_congress

In [4]:
members = create_members_df()
members

,icpsr,congress,party_code,bioname,state_abbrev,party_name
0,1,[82],[200],"AANDAHL, Fred George",ND,[Republican]
1,2,"[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...",[100],"ABBITT, Watkins Moorman",VA,[Democrat]
2,3,"[28, 29, 30]",[29],"ABBOTT, Amos",MA,[Whig]
3,4,"[50, 51, 52, 53, 54]",[100],"ABBOTT, Jo (Joseph)",TX,[Democrat]
4,5,"[15, 16, 17, 18]","[7777, 13]","ABBOT, Joel",GA,"[Crawford Republican, Democrat-Republican]"
...,...,...,...,...,...,...
12579,99910,"[107, 108, 109, 110]",[200],"BUSH, George Walker",USA,[Republican]
12580,99911,"[111, 112, 113, 114]",[100],"OBAMA, Barack",USA,[Democrat]
12581,99912,"[115, 116, 117]",[200],"TRUMP, Donald John",USA,[Republican]
12582,99913,[117],[100],"BIDEN, Joseph Robinette, Jr.",USA,[Democrat]


In [5]:
members.to_csv(dataset_folder+'members.csv', index=False)